# Download dataset


In [16]:
!curl -L -o data.tsv https://raw.githubusercontent.com/auwalsoe/encode_nlp_workshop_2023/main/data/papyrus_data.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8467k  100 8467k    0     0  9106k      0 --:--:-- --:--:-- --:--:-- 9105k


In [49]:
import pandas as pd

data = pd.read_csv('/content/data.tsv', delimiter = '\t')
#data = pd.read_csv('data.tsv', delimiter = '\t')

In [50]:
data.head()

,Unnamed: 0,translation,category,author,summary,keywords,originDate,provenance,num_words_in_translation
0,0,"kollesis obscures much of the text. 6 lines, o...",documentary text,unknown,unknown,papyri,Between 300 and 130 B.C.,unknown,24
1,3,"21 lines, on recto along the fibers; 2 lines (...",letter : conclusion : to maron,unknown,conclusion of a letter to maron about various ...,papyri,Late 2nd or 3rd century A.D.,unknown,24
2,1,3+ cols. (54+ lines) on recto along the fibers...,account : fragment,unknown,unknown,papyri,Late 2nd century B.C.,unknown,12
3,3,"10 lines, on verso across the fibers. on recto...",tax receipt,diodoros,"diodoros and partners, farmers of the crown ta...",papyri,181/182 or 213/214 A.D.,unknown,67
4,3,"49 lines, on verso across the fibers. recto em...",petition from harmiysis and the crown tenants ...,harmiysis,harmiysis and the crown tenants complain to kr...,papyri,Between 105 and 90 B.C.,unknown,240


In [51]:
data = data[data.provenance != 'unknown']
data =data[data.groupby('provenance')['provenance'].transform('count').ge(100)]


translations = data['translation'].values
provenance = data['provenance'].values



In [52]:
translations[:10]

array(['11 lines, on recto along the fibers. verso: account ',
       '19 cols. (315 lines) on recto along the fibers on verso: register of rents and taxes (p.tebt.1106) ',
       '5 lines, on recto along the fibers verso empty � (month) 12-16 in (canal) on behalf of tebtunis, herakl() son of pak() son of kronno(), his mother being heraklous. ',
       '16 lines, on recto along the fibers. on verso: some faint traces of ink. ',
       '9 cols. (127 lines) on recto along the fibers. on verso: decrees of king ptolemy viii euergetes ii : copy ',
       'large portions of papyrus still gummed on top of linen 11+ cols. (160+ lines) on recto along the fibers. on verso of one of smaller fragments remains of a text; remainder of verso empty or covered by linen. ',
       'badly preserved, text being fragmentary and mutilated. 9+ cols. (73+ lines) on verso across the fibers. recto: p.tebt.832 (account of crops and rentals) ',
       'out to conservation 13 dec 04 pjr 3 cols. (78 lines) on recto

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(translations, provenance, test_size=0.33, random_state=42)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()

X_train= tfidf_vect.fit_transform(X_train)
X_test = tfidf_vect.transform(X_test)

In [55]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [56]:
y_pred = model.predict(X_test)

In [58]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                                                               precision    recall  f1-score   support

                            Arsinoite nome, province of Egypt       0.84      1.00      0.91       889
                        Herkaleopolite nome province of Egypt       1.00      0.49      0.65        37
                                                 Kerkeosiris        0.82      0.77      0.80        61
Oxyrhynchos, Oxyrhinchite nome Middle Egypt province of Egypt       0.78      0.32      0.46       145
                                                    Tebtunis        1.00      0.24      0.39        46
                                     Thebes province of Egypt       1.00      0.94      0.97       214
                                                  Wadi Sarga        1.00      1.00      1.00        56
                                                     berenike       1.00      0.79      0.88        66

                                                     accuracy          

In [60]:
import gradio as gr

def find_my_provenance(text):
    text_tfidf = tfidf_vect.transform([text])
    return str(model.predict(text_tfidf)[0])
demo = gr.Interface(fn = find_my_provenance, inputs=gr.Textbox(lines=3,placeholder="Papyrus translation here"), outputs="text")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://26036452-a4fa-4e80.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
